# 06 Texte auswerten mit dem Computer

Ein wenig rechnen mit Wörtern: Welche Stichwörter beschreiben einen Text am besten?

In [ ]:
import math
from collections import Counter
from pathlib import Path
import re
from collections import Counter
from pathlib import Path
import re
    
def bag_of_words(text):
    # Convert to lowercase and extract words
    
    words = re.findall(r'\w+', text.lower())
    
    # Count frequencies and return as dict sorted by frequency
    word_counts = Counter(words)
    return dict(sorted(word_counts.items(), key=lambda item: item[1], reverse=True))

text1 = """
Was chatGPT (etc.) eigentlich ist: eine Benutzeroberfläche, unter der sich ein so 
genanntes “Generatives Sprachmodell” verbirgt, die eigentliche KI: eine Maschine, 
die wohlgeformte Texte produzieren kann. Das hat sie aus einer enormen Menge von 
menschengemachten Texten gelernt: Wie würden Menschen in einer vergleichbaren 
Situation wohl das nächste Wort wählen? Und dann wieder das nächste, und wieder 
das nächste.  
"""

text2 = """
Was sie nicht ist: Eine Wahrheitsmaschine, ein Weltgeist oder eine echte 
“Künstliche Intelligenz” im Sinne dessen, was KI-Forscher:innen erst noch 
erreichen wollen: chatGPT ist keine  “AGI” (“Artificial General Intelligence”). 
"""

print(bag_of_words(text1))

**Jetzt du:** Lass dir ein Programm schreiben, das die Häufigkeit der einzelnen Wörter angibt: Für ```text1```, für ```text2``` - und für beide Texte zusammen. 

## Topic Modeling: Die richtigen Schlagworte extrahieren


In [ ]:
print(bag_of_words(text1))
print(bag_of_words(text2))
print(bag_of_words(text1 + text2))

In [ ]:


def tf_idf_keywords(text, threshold=4):
    # Simple TF-IDF implementation
 
    
    # Tokenize and normalize
    words = re.findall(r'\w+', text.lower())
    
    # Calculate term frequency
    tf = Counter(words)
    
    # Calculate document frequency
    df = Counter()
    for doc in words:
        doc_words = set(re.findall(r'\w+', doc.lower()))
        for word in doc_words:
            df[word] += 1
    
    # Calculate TF-IDF scores
    tf_idf = {}
    N = len(words)
    
    for word, frequency in tf.items():
        if word in df:
            idf = math.log(N / df[word])
            val = frequency * idf
            if val >= threshold:
                tf_idf[word] = val
    
    # Return sorted keywords by weight
    return dict(sorted(tf_idf.items(), key=lambda x: x[1], reverse=True))


**Jetzt du**: Wende die tf_idf Funktion auf text1 und text2 an - und vergleiche mit dem Bag of words!

In [ ]:
print(bag_of_words(text1 + text2))
print(tf_idf_keywords(text1 + text2))

Okay... aber zu viel Müll: Wir müssen: 
- die **Stoppwörter** rausnehmen (ein, die, und...)
- die Wortstämme (Lemmata) extrahieren

Das machen wir mit einer Spezial-Bibliothek (die auch schon ein wenig KI benutzt): Spacy. 

Außerdem benutzen wir die Tabellen-Bibliothek pandas - und wir geben ihr einen anderen Namen. Details sind nicht so wichtig; einfach schauen: ähnliche Code-Schnipsel sieht man sehr oft!

In [ ]:
!pip install spacy
!pip install scikit.learn
!pip install pandas


In [ ]:
import spacy
from spacy.lang.de.examples import sentences 
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.cli import download
import pandas as pd

def load_model(model_name):
    try:
        nlp = spacy.load(model_name)
    except OSError:
        download(model_name)         # lädt das passende Paket ins aktuelle Python-Umfeld
        nlp = spacy.load(model_name)
    return nlp

# Load the spaCy model
nlp = load_model("de_core_news_sm")

# Sample text
text = text1 + text2 

# Lemmatization using spaCy
doc = nlp(text)
lemmatized_text = " ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])

print("Original text:")
print(text)
print("\nLemmatized text:")
print(lemmatized_text)

# TF-IDF calculation
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform([lemmatized_text])

# Get feature names (words)
feature_names = vectorizer.get_feature_names_out()

# Create a DataFrame for better visualization
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)
print("\nTF-IDF scores:")
print(tfidf_df)

Das ist schon sehr beeindruckend, aber noch vergleichsweise naiv: Wäre es nicht schöner, wenn wir statt dessen eine KI fragen, welches die wichtigsten Wörter in einem Text sind?

In [ ]:
!pip install mistralai

In [ ]:
from mistralai import Mistral

api_key = "AVtuDWTw4B0dS70w3nssuwFUnkhQuU0c"
model = "mistral-medium-latest"

client = Mistral(api_key=api_key)

def ask(prompt):
    chat_response = client.chat.complete(
        model= model,
        messages = [
            {
                "role": "user",
                "content": prompt,
            },
        ]
    )
    return chat_response.choices[0].message.content   


# Hier ergänzen: Das Modell soll selbst Keywords extrahieren

## Audio verschriftlichen

...und es wird noch cooler! Mit der richtigen Bibliothek kann man auch Audio in Text verwandeln: **Whisper** von OpenAI wurde ursprünglich dazu entwickelt, um (vor allem) Youtube-Videos zu transkribieren. Und setzt seitdem den Standard für saubere Transkription. 

In [ ]:
# Erst die Bibliothek holen
!pip install faster_whisper

In [ ]:
from faster_whisper import WhisperModel
import os

model = WhisperModel(
    "deepdml/faster-whisper-large-v3-turbo-ct2",  # CT2 model id
    device="cpu",                 # or "cuda"
    compute_type="int8",          # good for CPU; try "int8_float16" on CUDA
)

path="./daten/demoaudio.mp3"

if not os.path.exists(path):
    print(f"{path} gibt es leider nicht.")
    raise

segments, info = model.transcribe(
    path,
    language="de",                # force German
    beam_size=5,
    temperature=0.0,
    vad_filter=True,              # optional: basic VAD
)

text = "".join(s.text for s in segments)
print(text)

**Jetzt du**: Schreib eine Funktion, die Audios in Text umwandeln kann. Übergib der Funktion den Pfad zu einer Audio-Datei, die du in Text verwandeln möchtest. 

TIPP: Whisper kann nicht mit allen Dateitypen umgehen!